In [1]:
"""DL constants"""
from copy import copy
from functools import reduce
import pickle
import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from keras.layers import Add, Activation, Lambda, Input, Embedding
from keras.layers import Reshape, Flatten, Dot
from tensorflow.keras.models import Sequential, Model, load_model, save_model
from keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

TYPE_MAP = {0: "вторые блюда",
            1: "десерты",
            2: "завтраки",
            3: "закуски",
            4: "салаты",
            5: "соусы",
            6: "супы и первые блюда"}
CUISINES = {0: "европейская",
            1: "русская",
            2: "азиатская",
            3: "африканская",
            4: "мексиканская"}



2022-11-12 10:56:06.221505: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-12 10:56:06.522424: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-12 10:56:06.522475: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-12 10:56:06.594319: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-12 10:56:07.661377: W tensorflow/stream_executor/platform/de

In [2]:
def get_actual_items(ratings):
    """ get_actual_items """
    actual_items = list(set(ratings['recipeId'].values.tolist()))
    return actual_items


def get_actual_users(ratings):
    """ get_actual_users """
    actual_users = list(set(ratings['userId'].values.tolist()))
    return actual_users


def get_unique_user_map(ratings, actual_users):
    """ get_unique_user_map """
    user_enc = LabelEncoder()
    user_enc.fit(ratings['userId'].values)
    res = {el: user_enc.transform([el])[0] for el in actual_users}
    del user_enc
    return res


def get_unique_item_map(ratings, actual_items):
    """ get_unique_item_map """
    item_enc = LabelEncoder()
    item_enc.fit(ratings['recipeId'].values)
    res = {el: item_enc.transform([el])[0] for el in actual_items}
    del item_enc
    return res


def insert_users_and_recipes_into_df(df):
    """ insert users and recipes into dataframe"""
    user_enc = LabelEncoder()
    df['user'] = user_enc.fit_transform(df['userId'].values)
    item_enc = LabelEncoder()
    df['recipe'] = item_enc.fit_transform(df['recipeId'].values)
    del user_enc, item_enc


def load_users_and_ratings_as_x_y_df(users_csv="../data/users_ratings.csv"):
    """ load users and ratings as x, y, dataframe """
    ratings = pd.read_csv(users_csv)
    insert_users_and_recipes_into_df(ratings)
    ratings['rating'] = ratings['rating'].values.astype(np.float32)
    x = ratings[['user', 'recipe']].values
    y = ratings['rating'].values
    return x, y, ratings


def get_train_test_arrays(x_train, x_test):
    """ """
    x_train_array = [x_train[:, 0], x_train[:, 1]]
    x_test_array = [x_test[:, 0], x_test[:, 1]]
    return x_train_array, x_test_array


class EmbeddingLayer:
    """ Embedding Layer """
    def __init__(self, n_items, n_factors):
        self.n_items = n_items
        self.n_factors = n_factors

    def __call__(self, x):
        x = Embedding(self.n_items, self.n_factors, embeddings_initializer='he_normal',
                      embeddings_regularizer=l2(1e-6))(x)
        x = Reshape((self.n_factors,))(x)
        return x


def recommender_v2(n_users, n_items, n_factors, min_rating, max_rating):
    """ Recommender V2 """
    user = Input(shape=(1,))
    u = EmbeddingLayer(n_users, n_factors)(user)
    ub = EmbeddingLayer(n_users, 1)(user)
    movie = Input(shape=(1,))
    m = EmbeddingLayer(n_items, n_factors)(movie)
    mb = EmbeddingLayer(n_items, 1)(movie)
    x = Dot(axes=1)([u, m])
    x = Add()([x, ub, mb])
    x = Activation('sigmoid')(x)
    x = Lambda(lambda x: x * (max_rating - min_rating) + min_rating)(x)
    model = Model(inputs=[user, movie], outputs=x)
    opt = Adam(learning_rate=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model


def extract_and_save_embeddings(model):
    """ extract and save users embeddings """
    w1 = None
    for layer in model.layers:
        num = str(layer.name)[-2:]
        if isinstance(layer, Embedding):  # (1, 3076, 60)
            w1 = layer
            print(f"layer {num}", np.array(w1.get_weights()).shape)
            break
    users_embedding = np.array(w1.get_weights())
    with open("./users_embedding.pkl", "wb") as out_file:
        pickle.dump(users_embedding, out_file)
    return users_embedding


def l2_norm(vect):
    """ l2 norm """
    tmp = [x * x for x in vect.tolist()]
    res = np.sqrt(reduce(lambda x, y: x + y, tmp))
    return res


def cosine_sim(v1, v2, eps1=1e-4, eps2=1e-4):
    """ cosine similarity """
    div = (l2_norm(v1) + eps1) * (l2_norm(v2) + eps2)
    res = np.dot(v1, v2) / div
    return res


def find_n_closest(id_, ids, users_dict, embed, n=100):
    """ find the N-nearest vectors """
    minus_one = copy(ids)
    minus_one.remove(id_)
    k = users_dict[id_]
    encoded_ids = list(map(lambda x: users_dict[x], ids))
    vect = embed[0][k][...]
    res = [(i, cosine_sim(vect, embed[0][encoded_ids[i]][...])) for i in range(len(minus_one))]
    sort = sorted(res, key=lambda x: x[1], reverse=True)[:n]
    similars = [elem[1] for elem in sort]
    actuals = [minus_one[elem[0]] for elem in sort]
    return actuals, similars


def get_users_items_matrix_and_popularities(ratings):
    """ Construct sparse matrix """
    X = ratings[['user', 'recipe']].values
    y = ratings['rating'].values
    n_users = ratings['user'].nunique()
    n_items = ratings['recipe'].nunique()
    mat = sp.dok_matrix((n_users + 1, n_items + 1), dtype=np.float32)
    for i in range(len(y)):
        if y[i] > 0:
            mat[X[i][0], X[i][1]] = y[i]
    popular = np.zeros((n_items,))
    for i in range(len(y)):
        popular[X[i][1]] += 1
    return mat, popular


def average_rating(k, ratings):
    """ average_rating """
    vals = dict(ratings[k])
    summ = reduce(lambda x, y: x + y, vals.values()) / ratings.shape[0]
    return summ


class CFRecommender:
    """ Recommender of dishes using collaborative filtration """
    def __init__(self, users_csv="../data/users_ratings.csv", n_factors=60):
        x, y, ratings = load_users_and_ratings_as_x_y_df(users_csv=users_csv)
        self.ratings = ratings
        X_train, X_test, self.y_train, self.y_test = train_test_split(x,
                                                                      y,
                                                                      test_size=0.1,
                                                                      random_state=17)
        self.X_train_array, self.X_test_array = get_train_test_arrays(X_train,
                                                                      X_test)
        self.n_users = ratings['user'].nunique()
        self.n_items = ratings['recipe'].nunique()
        self.n_factors = n_factors
        self.actual_items = get_actual_items(self.ratings)
        self.model = recommender_v2(self.n_users, self.n_items, n_factors,
                               min(self.ratings['rating']), max(self.ratings['rating']))
        self.users_embedding = None
        self.matr = None
        self.popularity = None
        self.closest_users = None
        self.similarities = None
        self.closest_users = None
        self.similarities = None

    def get_pretrained_embeddings(self, pickle_file="../models/users_embedding.pkl"):
        """ get pretrained embeddings """
        with open(pickle_file, "rb") as pkl_file:
            self.users_embedding = pickle.load(pkl_file)

    def train_model_and_get_embeddings(self, epochs=2222):
        """ train model and get embeddings """
        
        hist = self.model.fit(x=self.X_train_array, y=self.y_train,
                  batch_size=128,
                  epochs=epochs,
                  shuffle=True,
                  verbose=1,
                  validation_data=(self.X_test_array, self.y_test))
        self.model.save("./cf_short_net_2222ep.h5")
        self.users_embedding = extract_and_save_embeddings(self.model)
        return hist

    def find_rating_for_dish(self, user_id, item_id, user_enc, item_enc):
        """ Вычисление оценки рейтинга по схожим клиентам
            (User based Collaborative Filtering)
            arguments user_id and item_id
        """
        dish = item_enc[item_id]
        avg_u = average_rating(user_enc[user_id], self.matr)
        weighted_sum = 0.
        sum_ = 0.
        for k in range(len(self.closest_users)):
            j = user_enc[self.closest_users[k]]
            if self.matr[j, dish] > 0:
                rate = self.matr[j, dish]
                avg_v = average_rating(j, self.matr)
                weighted_sum += self.similarities[k] * (rate - avg_v)
                sum_ += self.similarities[k]
        if sum_ > 0:
            return int(avg_u + (weighted_sum / sum_))
        return 1

    def find_100_closest_ratings_for_user(self, user_id, user_dict, item_dict):
        """ find 100 closest ratings for user """
        del self.matr, self.popularity
        self.matr, self.popularity = get_users_items_matrix_and_popularities(
            self.ratings)
        all_users = get_actual_users(self.ratings)
        self.closest_users, self.similarities = find_n_closest(user_id,
                                                               all_users,
                                                               user_dict,
                                                               self.users_embedding,
                                                               n=100)
        actual_items = get_actual_items(self.ratings)
        rating_list = [self.find_rating_for_dish(user_id,
                                                 i,
                                                 user_dict,
                                                 item_dict) for i in actual_items]
        return rating_list

    def find_top_items_for_rating_5(self, rating_list):
        """ find top 9 for ratings 5 """
        zipped = zip(rating_list, self.popularity, list(range(len(rating_list))))
        sorted_ratings = sorted(list(zipped), key=lambda x: x[1], reverse=True)
        best_rate5 = [elem for elem in sorted_ratings if elem[0] == 5]
        top_9_rating5 = [self.actual_items[el[2]] for el in best_rate5]
        return top_9_rating5


In [4]:
def recommender_v2(n_users, n_items, n_factors, min_rating, max_rating):
    """ Recommender V2 """
    user = Input(shape=(1,))
    u = EmbeddingLayer(n_users, n_factors)(user)
    ub = EmbeddingLayer(n_users, 1)(user)
    movie = Input(shape=(1,))
    m = EmbeddingLayer(n_items, n_factors)(movie)
    mb = EmbeddingLayer(n_items, 1)(movie)
    x = Dot(axes=1)([u, m])
    x = Add()([x, ub, mb])
    x = Activation('sigmoid')(x)
    x = Lambda(lambda x: x * (max_rating - min_rating) + min_rating)(x)
    model = Model(inputs=[user, movie], outputs=x)
    opt = Adam(learning_rate=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

user_id=2
load_pretrained=False

## do calculations and return top items with rating 5 ##
recommender = CFRecommender(users_csv="../data/users_ratings.csv",
                            n_factors=60)
hist = recommender.train_model_and_get_embeddings(epochs=2000)
user_dict = get_unique_user_map(recommender.ratings,
                                get_actual_users(recommender.ratings))
item_dict = get_unique_item_map(recommender.ratings,
                                recommender.actual_items)
rating_list = recommender.find_100_closest_ratings_for_user(user_id,
                                                            user_dict,
                                                            item_dict)
top_items_with_rating5 = recommender.find_top_items_for_rating_5(
    rating_list)
    


Epoch 1/2000
37/37 [==============================] - 2s 14ms/step - loss: 3.1496 - val_loss: 3.0494
Epoch 2/2000
37/37 [==============================] - 0s 7ms/step - loss: 3.0221 - val_loss: 2.9843
Epoch 3/2000
37/37 [==============================] - 0s 8ms/step - loss: 2.8875 - val_loss: 2.9217
Epoch 4/2000
37/37 [==============================] - 0s 8ms/step - loss: 2.7299 - val_loss: 2.8608
Epoch 5/2000
37/37 [==============================] - 0s 8ms/step - loss: 2.5406 - val_loss: 2.8023
Epoch 6/2000
37/37 [==============================] - 0s 7ms/step - loss: 2.3154 - val_loss: 2.7485
Epoch 7/2000
37/37 [==============================] - 0s 7ms/step - loss: 2.0640 - val_loss: 2.6978
Epoch 8/2000
37/37 [==============================] - 0s 7ms/step - loss: 1.8025 - val_loss: 2.6517
Epoch 9/2000
37/37 [==============================] - 0s 7ms/step - loss: 1.5489 - val_loss: 2.6110
Epoch 10/2000
37/37 [==============================] - 0s 8ms/step - loss: 1.3159 - val_loss: 2.574

37/37 [==============================] - 0s 7ms/step - loss: 0.0141 - val_loss: 2.3666
Epoch 83/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0140 - val_loss: 2.3666
Epoch 84/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0139 - val_loss: 2.3665
Epoch 85/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0138 - val_loss: 2.3662
Epoch 86/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0136 - val_loss: 2.3667
Epoch 87/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0135 - val_loss: 2.3662
Epoch 88/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0135 - val_loss: 2.3667
Epoch 89/2000
37/37 [==============================] - 0s 8ms/step - loss: 0.0134 - val_loss: 2.3664
Epoch 90/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0133 - val_loss: 2.3664
Epoch 91/2000
37/37 [==============================] - 0s 9ms/step - loss: 0.0132 - val_loss: 2.3660
Epoc

37/37 [==============================] - 0s 7ms/step - loss: 0.0118 - val_loss: 2.3371
Epoch 163/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0117 - val_loss: 2.3366
Epoch 164/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0117 - val_loss: 2.3354
Epoch 165/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0117 - val_loss: 2.3348
Epoch 166/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0117 - val_loss: 2.3337
Epoch 167/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0117 - val_loss: 2.3326
Epoch 168/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0118 - val_loss: 2.3315
Epoch 169/2000
37/37 [==============================] - 0s 9ms/step - loss: 0.0116 - val_loss: 2.3310
Epoch 170/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0116 - val_loss: 2.3302
Epoch 171/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0116 - val_loss: 2.

37/37 [==============================] - 0s 7ms/step - loss: 0.0114 - val_loss: 2.2161
Epoch 243/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0113 - val_loss: 2.2139
Epoch 244/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0114 - val_loss: 2.2113
Epoch 245/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0113 - val_loss: 2.2083
Epoch 246/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0113 - val_loss: 2.2066
Epoch 247/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0114 - val_loss: 2.2044
Epoch 248/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0113 - val_loss: 2.2006
Epoch 249/2000
37/37 [==============================] - 0s 8ms/step - loss: 0.0113 - val_loss: 2.1980
Epoch 250/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0112 - val_loss: 2.1960
Epoch 251/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0113 - val_loss: 2.

37/37 [==============================] - 0s 7ms/step - loss: 0.0109 - val_loss: 1.9791
Epoch 323/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0109 - val_loss: 1.9752
Epoch 324/2000
37/37 [==============================] - 0s 8ms/step - loss: 0.0109 - val_loss: 1.9720
Epoch 325/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0108 - val_loss: 1.9691
Epoch 326/2000
37/37 [==============================] - 0s 8ms/step - loss: 0.0107 - val_loss: 1.9654
Epoch 327/2000
37/37 [==============================] - 0s 9ms/step - loss: 0.0107 - val_loss: 1.9616
Epoch 328/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0107 - val_loss: 1.9588
Epoch 329/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0108 - val_loss: 1.9552
Epoch 330/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0109 - val_loss: 1.9513
Epoch 331/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0110 - val_loss: 1.

...

37/37 [==============================] - 0s 8ms/step - loss: 0.0091 - val_loss: 1.0728
Epoch 1674/2000
37/37 [==============================] - 0s 10ms/step - loss: 0.0090 - val_loss: 1.0734
Epoch 1675/2000
37/37 [==============================] - 0s 10ms/step - loss: 0.0091 - val_loss: 1.0731
Epoch 1676/2000
37/37 [==============================] - 0s 9ms/step - loss: 0.0091 - val_loss: 1.0732
Epoch 1677/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0090 - val_loss: 1.0729
Epoch 1678/2000
37/37 [==============================] - 0s 10ms/step - loss: 0.0090 - val_loss: 1.0727
Epoch 1679/2000
37/37 [==============================] - 0s 8ms/step - loss: 0.0090 - val_loss: 1.0727
Epoch 1680/2000
37/37 [==============================] - 0s 8ms/step - loss: 0.0091 - val_loss: 1.0728
Epoch 1681/2000
37/37 [==============================] - 0s 9ms/step - loss: 0.0091 - val_loss: 1.0723
Epoch 1682/2000
37/37 [==============================] - 0s 9ms/step - loss: 0.0092 - 

37/37 [==============================] - 0s 10ms/step - loss: 0.0092 - val_loss: 1.0666
Epoch 1753/2000
37/37 [==============================] - 0s 9ms/step - loss: 0.0091 - val_loss: 1.0660
Epoch 1754/2000
37/37 [==============================] - 0s 8ms/step - loss: 0.0091 - val_loss: 1.0668
Epoch 1755/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0090 - val_loss: 1.0655
Epoch 1756/2000
37/37 [==============================] - 0s 8ms/step - loss: 0.0091 - val_loss: 1.0663
Epoch 1757/2000
37/37 [==============================] - 0s 10ms/step - loss: 0.0090 - val_loss: 1.0653
Epoch 1758/2000
37/37 [==============================] - 0s 8ms/step - loss: 0.0090 - val_loss: 1.0666
Epoch 1759/2000
37/37 [==============================] - 0s 9ms/step - loss: 0.0091 - val_loss: 1.0652
Epoch 1760/2000
37/37 [==============================] - 0s 8ms/step - loss: 0.0091 - val_loss: 1.0660
Epoch 1761/2000
37/37 [==============================] - 0s 10ms/step - loss: 0.0092 - 

37/37 [==============================] - 0s 9ms/step - loss: 0.0091 - val_loss: 1.0612
Epoch 1832/2000
37/37 [==============================] - 0s 9ms/step - loss: 0.0090 - val_loss: 1.0609
Epoch 1833/2000
37/37 [==============================] - 0s 11ms/step - loss: 0.0091 - val_loss: 1.0610
Epoch 1834/2000
37/37 [==============================] - 0s 8ms/step - loss: 0.0091 - val_loss: 1.0602
Epoch 1835/2000
37/37 [==============================] - 0s 7ms/step - loss: 0.0090 - val_loss: 1.0597
Epoch 1836/2000
37/37 [==============================] - 0s 8ms/step - loss: 0.0091 - val_loss: 1.0606
Epoch 1837/2000
37/37 [==============================] - 0s 8ms/step - loss: 0.0091 - val_loss: 1.0607
Epoch 1838/2000
37/37 [==============================] - 0s 8ms/step - loss: 0.0091 - val_loss: 1.0602
Epoch 1839/2000
37/37 [==============================] - 0s 8ms/step - loss: 0.0091 - val_loss: 1.0599
Epoch 1840/2000
37/37 [==============================] - 0s 9ms/step - loss: 0.0091 - va

37/37 [==============================] - 0s 11ms/step - loss: 0.0090 - val_loss: 1.0562
Epoch 1911/2000
37/37 [==============================] - 0s 9ms/step - loss: 0.0090 - val_loss: 1.0556
Epoch 1912/2000
37/37 [==============================] - 0s 8ms/step - loss: 0.0090 - val_loss: 1.0558
Epoch 1913/2000
37/37 [==============================] - 0s 11ms/step - loss: 0.0090 - val_loss: 1.0559
Epoch 1914/2000
37/37 [==============================] - 1s 15ms/step - loss: 0.0090 - val_loss: 1.0554
Epoch 1915/2000
37/37 [==============================] - 0s 9ms/step - loss: 0.0091 - val_loss: 1.0556
Epoch 1916/2000
37/37 [==============================] - 0s 9ms/step - loss: 0.0090 - val_loss: 1.0556
Epoch 1917/2000
37/37 [==============================] - 0s 11ms/step - loss: 0.0091 - val_loss: 1.0554
Epoch 1918/2000
37/37 [==============================] - 1s 15ms/step - loss: 0.0091 - val_loss: 1.0551
Epoch 1919/2000
37/37 [==============================] - 0s 12ms/step - loss: 0.0090

37/37 [==============================] - 0s 7ms/step - loss: 0.0090 - val_loss: 1.0516
Epoch 1990/2000
37/37 [==============================] - 0s 8ms/step - loss: 0.0090 - val_loss: 1.0524
Epoch 1991/2000
37/37 [==============================] - 0s 8ms/step - loss: 0.0091 - val_loss: 1.0515
Epoch 1992/2000
37/37 [==============================] - 0s 9ms/step - loss: 0.0091 - val_loss: 1.0519
Epoch 1993/2000
37/37 [==============================] - 0s 10ms/step - loss: 0.0090 - val_loss: 1.0515
Epoch 1994/2000
37/37 [==============================] - 0s 8ms/step - loss: 0.0091 - val_loss: 1.0522
Epoch 1995/2000
37/37 [==============================] - 0s 8ms/step - loss: 0.0090 - val_loss: 1.0511
Epoch 1996/2000
37/37 [==============================] - 0s 8ms/step - loss: 0.0090 - val_loss: 1.0519
Epoch 1997/2000
37/37 [==============================] - 0s 9ms/step - loss: 0.0091 - val_loss: 1.0509
Epoch 1998/2000
37/37 [==============================] - 0s 8ms/step - loss: 0.0090 - va

In [5]:
history = recommender.evaluate(x=recommender.X_test_array,
                            y=recommender.y_test)

top_items_with_rating5[:10]


[647, 473, 1524, 2840, 3294, 1258, 860, 3806, 665, 638]